In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
#df = df[df["asset"]=="NVDA"]
#display(df[df["asset"]=="NVDA"])

In [3]:
returns = df['Total Return [%]'] / 100
cumulative_return = (1 + returns).prod() - 1
N = len(returns)  # number of periods
annualized_return = (1 + cumulative_return)**(12 / N) - 1
annualized_volatility = returns.std() * np.sqrt(12)
sharpe_annual = annualized_return / annualized_volatility
print(sharpe_annual)

0.1422256849843752


In [4]:
def compute_annualized_sharpe(df):
    results = []

    for asset, group in df.groupby("asset"):
        # Convert returns from % to decimals
        returns = group['Total Return [%]'] / 100
        benchmark_returns = group['Benchmark Return [%]'] / 100
        N = len(returns)

        if N < 2:
            continue  # Skip assets with insufficient data

        # Strategy Sharpe
        cumulative_return = (1 + returns).prod() - 1
        annualized_return = (1 + cumulative_return)**(12 / N) - 1
        annualized_volatility = returns.std() * np.sqrt(12)
        sharpe_strategy = annualized_return / annualized_volatility if annualized_volatility > 0 else np.nan

        # Benchmark Sharpe
        cumulative_bench = (1 + benchmark_returns).prod() - 1
        annualized_bench_return = (1 + cumulative_bench)**(12 / N) - 1
        annualized_bench_vol = benchmark_returns.std() * np.sqrt(12)
        sharpe_benchmark = annualized_bench_return / annualized_bench_vol if annualized_bench_vol > 0 else np.nan

        results.append({
            'asset': asset,
            'strategy_sharpe': sharpe_strategy,
            'benchmark_sharpe': sharpe_benchmark
        })

    return pd.DataFrame(results)

In [7]:
df = pd.read_csv("vectorbt_reports/per_asset_expanding_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
sharpe_df = compute_annualized_sharpe(df)
display(sharpe_df.sort_values(by='strategy_sharpe', ascending=False))

,asset,strategy_sharpe,benchmark_sharpe
9,TSLA,0.852583,0.954963
1,AMZN,0.081659,0.382994
3,BTC=F,-0.019230,0.580419
2,BTC-USD,-0.203990,0.654541
5,GC=F,-0.411872,2.864663
6,GOOGL,-0.578822,-0.416418
8,NVDA,-0.808829,-0.209887
0,AAPL,-0.970889,0.652176
4,CL=F,-1.052528,-0.187782
7,MSFT,-1.245813,-0.405248


In [8]:
for asset_name in df["asset"].unique():
    asset_df = df[df["asset"] == asset_name]
    print(f"Asset: {asset_name}")
    display(asset_df)

Asset: AAPL


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2024-07-01,20 days,0.244181,11.709530,100.0,49.925362,3.419461,9 days,3,...,-0.668533,6 days,5 days,1.515711,17.221268,0.305004,1.331032,1.051996,0.470377,AAPL
10,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,3.118142,2.459055,100.0,30.634283,4.721883,11 days 00:00:00,2,...,NaN,12 days 00:00:00,NaN,NaN,322.125979,2.266365,14.069306,1.449063,3.249415,AAPL
20,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,5.167669,3.115993,100.0,9.990010,0.874132,2 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,6.702453,140.099057,3.464486,17.956919,AAPL
30,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,-3.772958,-0.528387,100.0,47.903914,6.679033,19 days 00:00:00,3,...,-3.399966,NaN,2 days 12:00:00,0.000000,-333.951634,-2.758863,-7.551284,0.564494,-3.767346,AAPL
40,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-4.861173,0.287553,100.0,57.971789,4.766034,21 days 00:00:00,3,...,-1.641383,NaN,4 days 08:00:00,0.000000,-162.039100,-4.698475,-11.803152,0.408052,-5.136934,AAPL
50,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-0.373799,0.539220,100.0,49.023439,3.304101,15 days 00:00:00,3,...,-1.618389,NaN,1 days 12:00:00,0.000000,-160.370249,-0.504156,-1.823247,0.906042,-0.681076,AAPL
60,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,4.430279,7.693926,100.0,41.305131,2.337698,7 days 00:00:00,2,...,NaN,7 days 00:00:00,NaN,NaN,221.513944,5.206242,51.584209,2.448570,8.159433,AAPL
70,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-7.565943,-8.858048,100.0,47.648996,7.565943,8 days 00:00:00,3,...,-3.812185,NaN,1 days 00:00:00,0.000000,-373.675464,-6.810296,-10.536189,0.000000,-6.579111,AAPL
80,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-5.050822,4.567998,100.0,29.091384,5.392659,19 days 00:00:00,2,...,-4.497721,NaN,4 days 00:00:00,0.000000,-449.322785,-4.327216,-10.695786,0.365594,-4.561331,AAPL
90,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-5.060565,-9.344320,100.0,28.764879,6.716227,21 days 00:00:00,2,...,-6.131873,NaN,6 days 00:00:00,0.000000,-612.574691,-2.859982,-8.358568,0.588244,-3.286417,AAPL


Asset: AMZN


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
1,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,2.373629,10.575306,100.0,51.059740,2.520152,10 days 00:00:00,3,...,-0.385755,8 days 00:00:00,2 days 00:00:00,7.245603,123.800128,2.203014,21.204262,1.489221,3.478840,AMZN
11,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-7.852674,-5.182556,100.0,46.594602,11.615233,18 days 00:00:00,3,...,-5.102037,NaN,6 days 12:00:00,0.000000,-498.528121,-5.012455,-6.392676,0.442891,-5.852842,AMZN
21,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,-8.494792,-4.535242,100.0,37.072966,10.458867,22 days 00:00:00,2,...,-10.394097,13 days 00:00:00,2 days 00:00:00,0.181912,-424.739575,-3.035616,-7.224165,0.512188,-3.458374,AMZN
31,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,2.487063,5.305323,100.0,39.413987,4.175035,6 days 00:00:00,2,...,-4.179210,14 days 00:00:00,4 days 00:00:00,1.595699,124.353162,1.571146,13.549723,1.237287,2.397364,AMZN
41,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-2.247795,2.415070,100.0,48.601349,3.784538,18 days 00:00:00,3,...,-1.853300,NaN,5 days 12:00:00,0.000000,-184.416122,-1.811783,-8.302486,0.703929,-2.067139,AMZN
51,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,12.842176,7.813239,100.0,32.109977,3.284389,8 days 00:00:00,2,...,NaN,12 days 00:00:00,NaN,NaN,1059.972554,5.394365,195.545728,2.484048,12.771520,AMZN
61,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,5.863392,7.629035,100.0,20.586946,5.422448,8 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,586.339194,3.464747,33.727508,1.779107,5.227399,AMZN
71,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,5.298772,6.380477,100.0,9.990010,1.303086,2 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,6.184607,141.895157,4.387577,19.139129,AMZN
81,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-2.206630,-8.949958,100.0,29.418468,5.522534,15 days 00:00:00,2,...,-2.860664,NaN,9 days 00:00:00,0.000000,-285.780582,-1.432194,-5.602566,0.753472,-1.700682,AMZN
91,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-4.360339,-10.090975,100.0,49.103186,6.314717,8 days 00:00:00,3,...,-1.443682,NaN,1 days 12:00:00,0.000000,-143.192200,-2.391724,-8.030894,0.614782,-2.979242,AMZN


Asset: BTC-USD


...

,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
7,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,4.003038,10.449319,100.0,40.353252,1.401555,2 days 00:00:00,2,...,-0.237698,13 days 00:00:00,2 days 00:00:00,17.857693,200.151921,5.242374,74.693354,2.360538,10.757452,MSFT
17,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-8.928048,-8.403215,100.0,47.832945,10.948565,18 days 00:00:00,3,...,-3.572484,NaN,7 days 00:00:00,0.000000,-350.544095,-6.865669,-7.198100,0.320705,-7.164542,MSFT
27,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,2.636152,-0.289229,100.0,9.990010,3.342745,8 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,2.922717,15.293844,1.597493,4.432063,MSFT
37,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,1.125271,2.608231,100.0,29.581626,2.432244,6 days 00:00:00,2,...,-2.044060,NaN,1 days 00:00:00,0.000000,-204.201799,1.190189,9.314930,1.192609,1.926165,MSFT
47,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-1.406361,0.383459,100.0,67.705238,4.458518,21 days 00:00:00,4,...,-1.502398,NaN,2 days 08:00:00,0.000000,-148.617270,-1.455453,-4.697005,0.764857,-2.065115,MSFT
57,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-6.622546,-2.074319,100.0,47.275680,7.581357,20 days 00:00:00,3,...,-3.776128,NaN,4 days 12:00:00,0.000000,-372.863265,-3.505243,-8.958293,0.464700,-3.811468,MSFT
67,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,3.083200,1.669581,100.0,20.308649,3.852345,7 days 00:00:00,1,...,NaN,13 days 00:00:00,NaN,NaN,308.319967,2.638339,19.222576,1.624994,3.437693,MSFT
77,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,1.833378,2.290331,100.0,9.990010,2.719884,2 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,1.759012,16.376515,1.574469,3.843912,MSFT
87,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-11.214402,-8.015001,100.0,46.727992,11.620875,21 days 00:00:00,3,...,-4.526852,NaN,1 days 12:00:00,0.000000,-446.113302,-6.842071,-7.409244,0.149283,-6.698250,MSFT
97,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-6.871206,-5.236040,100.0,48.653523,6.871206,21 days 00:00:00,3,...,-2.214724,NaN,2 days 00:00:00,0.000000,-218.807935,-6.058421,-9.850830,0.330017,-6.345061,MSFT


Asset: NVDA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
8,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,5.865311,8.086959,100.0,41.572972,6.738879,8 days 00:00:00,2,...,NaN,6 days 00:00:00,NaN,NaN,293.265532,2.830662,27.152773,1.621435,5.108975,NVDA
18,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-15.362350,-5.856803,100.0,45.293445,19.441296,15 days 00:00:00,3,...,-7.962904,NaN,5 days 12:00:00,0.000000,-763.885596,-5.378188,-4.820477,0.348199,-5.660427,NVDA
28,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,0.260382,2.008209,100.0,38.653996,9.636611,9 days 00:00:00,2,...,-6.867397,13 days 00:00:00,1 days 00:00:00,1.037954,13.019114,0.359885,0.437199,1.058271,0.510125,NVDA
38,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,12.191235,1.700594,100.0,9.990010,4.843039,7 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,4.699536,147.864435,2.384347,13.088979,NVDA
48,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,5.491187,16.312580,100.0,60.338799,4.876467,6 days 00:00:00,3,...,-3.852472,6 days 12:00:00,1 days 00:00:00,2.426792,183.039566,2.377851,29.275337,1.485502,3.522808,NVDA
58,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,-2.502010,-4.184073,100.0,49.764499,8.284628,14 days 00:00:00,3,...,-3.638499,8 days 00:00:00,3 days 00:00:00,0.346826,-120.225617,-1.223394,-4.142706,0.803938,-1.741719,NVDA
68,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,-2.073322,-0.896930,100.0,49.305320,8.203006,16 days 00:00:00,3,...,-3.828553,6 days 00:00:00,2 days 00:00:00,0.053346,-181.404816,-1.095659,-3.873578,0.829793,-1.676125,NVDA
78,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-13.317756,-13.870104,100.0,38.867760,22.022358,13 days 00:00:00,2,...,-14.193695,6 days 00:00:00,5 days 00:00:00,0.070112,-665.887815,-2.612433,-4.290519,0.595081,-2.900684,NVDA
88,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,5.061963,10.859653,100.0,9.990010,9.621014,4 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,2.329120,13.188031,1.447299,3.654689,NVDA
98,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,-20.027716,-16.461000,100.0,52.909460,19.947744,22 days 00:00:00,3,...,-7.158197,NaN,4 days 08:00:00,0.000000,-667.590535,-5.456780,-4.868621,0.344847,-5.877995,NVDA


Asset: TSLA


,Start,End,Period,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
9,2024-06-03 00:00:00,2024-07-01 00:00:00,20 days 00:00:00,11.128622,19.042492,100.0,29.851621,3.325209,5 days 00:00:00,2,...,-0.692735,NaN,4 days 00:00:00,0.000000,-69.204314,4.750423,176.229064,2.870912,13.953168,TSLA
19,2024-07-01 00:00:00,2024-07-31 00:00:00,22 days 00:00:00,-2.356342,10.583249,100.0,70.763122,14.139436,15 days 00:00:00,4,...,-3.058711,6 days 00:00:00,1 days 12:00:00,0.640056,-75.289946,-0.479823,-2.310815,0.905434,-0.627243,TSLA
29,2024-07-31 00:00:00,2024-08-30 00:00:00,23 days 00:00:00,2.918781,-7.739047,100.0,9.990010,7.847003,7 days 00:00:00,1,...,NaN,NaN,NaN,NaN,NaN,1.164760,7.374086,1.220348,1.870971,TSLA
39,2024-08-30 00:00:00,2024-09-27 00:00:00,20 days 00:00:00,5.118948,21.647747,100.0,28.264257,8.720137,12 days 00:00:00,2,...,-8.637486,NaN,1 days 00:00:00,0.000000,-862.885761,1.868876,17.052651,1.419356,2.765210,TSLA
49,2024-09-30 00:00:00,2024-10-29 00:00:00,22 days 00:00:00,-8.628634,-0.806488,100.0,28.350244,8.537263,21 days 00:00:00,2,...,-8.207121,NaN,3 days 00:00:00,0.000000,-819.892214,-5.458968,-9.092244,0.275392,-6.194006,TSLA
59,2024-10-29 00:00:00,2024-11-27 00:00:00,22 days 00:00:00,21.993761,28.271435,100.0,46.414057,1.246252,6 days 00:00:00,2,...,NaN,2 days 12:00:00,NaN,NaN,1099.688043,6.926798,2091.529288,15.273251,67.029079,TSLA
69,2024-11-29 00:00:00,2024-12-27 00:00:00,20 days 00:00:00,21.918403,25.060841,100.0,22.194054,1.686144,2 days 00:00:00,1,...,NaN,9 days 00:00:00,NaN,NaN,2191.840349,9.029348,2148.034132,7.244966,38.883103,TSLA
79,2024-12-30 00:00:00,2025-01-27 00:00:00,18 days 00:00:00,-10.881625,-4.853743,100.0,56.898775,10.924888,13 days 00:00:00,3,...,-3.761260,NaN,3 days 00:00:00,0.000000,-362.720845,-7.633224,-8.268235,0.221789,-8.085385,TSLA
89,2025-01-27 00:00:00,2025-02-26 00:00:00,22 days 00:00:00,-13.787024,-26.778297,100.0,64.605289,14.622247,17 days 00:00:00,4,...,-4.796997,NaN,3 days 00:00:00,0.000000,-456.693705,-7.263052,-6.255345,0.167056,-7.153249,TSLA
99,2025-02-26 00:00:00,2025-03-28 00:00:00,23 days 00:00:00,10.505351,-9.370702,100.0,39.178904,10.048871,5 days 00:00:00,2,...,-9.372969,4 days 00:00:00,4 days 00:00:00,2.121934,525.267574,2.753101,38.617391,1.837715,5.711084,TSLA


In [3]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)


,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-04-01,2025-04-01,252 days,10000.0,10721.883384,7.218834,31.265073,100.0,328.908744,17.037744,...,-2.375918,24 days 06:00:00,4 days 02:24:00,1.251347,48.688060,0.590394,0.623492,1.126426,0.915874,AAPL
1,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,10491.054606,4.910546,6.188869,100.0,349.808675,18.495673,...,-2.753795,21 days 00:00:00,2 days 08:43:38.181818181,1.157118,28.885565,0.388929,0.388746,1.072726,0.535029,AMZN
2,2024-04-01 00:00:00,2025-04-01 00:00:00,366 days 00:00:00,10000.0,7274.338275,-27.256617,22.190167,100.0,487.351735,34.495567,...,-3.332668,18 days 18:00:00,5 days 21:49:05.454545454,0.598333,-104.553361,-0.711268,-0.788314,0.871555,-1.026674,BTC-USD
3,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,7587.859227,-24.121408,21.270247,100.0,307.765811,46.649765,...,-4.052059,10 days 00:00:00,4 days 16:36:55.384615384,0.538093,-150.758798,-0.830316,-0.704172,0.824010,-1.119607,BTC=F
4,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,7856.763089,-21.432369,-14.944454,100.0,443.583593,28.547728,...,-2.533963,13 days 00:00:00,2 days 16:00:00,0.359421,-103.265213,-1.101587,-1.029486,0.827681,-1.460549,CL=F
5,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,11676.175660,16.761757,39.454500,100.0,289.481466,5.501840,...,-1.226264,15 days 10:17:08.571428571,6 days 20:00:00,2.855815,112.913693,1.625015,4.553550,1.325548,2.226771,GC=F
6,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,6802.639334,-31.973607,1.016144,100.0,333.089321,32.959059,...,-3.667618,9 days 18:00:00,4 days 09:36:00,0.297916,-173.468734,-2.024770,-1.297574,0.643346,-2.449228,GOOGL
7,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,9357.608399,-6.423916,-9.981865,100.0,301.430275,20.653183,...,-2.027989,35 days 16:00:00,5 days 02:10:54.545454545,0.729166,-45.216714,-0.351711,-0.443944,0.937761,-0.445826,MSFT
8,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,9173.209405,-8.267906,21.897240,100.0,333.679776,27.363468,...,-5.848730,8 days 03:25:42.857142857,4 days 04:48:00,0.854920,-48.634741,-0.131419,-0.429400,0.972280,-0.176850,NVDA
9,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,12880.047557,28.800476,53.213098,100.0,478.206041,25.077173,...,-4.754850,9 days 03:25:42.857142857,2 days 01:42:51.428571428,1.372193,137.145122,0.988293,1.765746,1.251895,1.649825,TSLA
